In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
print(pd.__version__)

1.3.5


In [ ]:
%cd /content/drive/MyDrive/BKMS Team Project/Recommendation

/content/drive/.shortcut-targets-by-id/1iIrNfxOKaEFXk_J5B-QHiYTGWzQMdVmx/BKMS Team Project/Recommendation


In [ ]:
# Install requirements
!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev
# PostgreSQL 패키지
!pip install psycopg2-binary
# neo4j 패키지 설치
!pip install neo4j

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 https://develo

In [ ]:
import joblib
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# import User_Recommend packages
import pandas as pd
import psycopg2

# neo4j 드라이버 import 및 함수 정의
from neo4j import GraphDatabase

## Content_Recommend packages
# import packages
import numpy as np

In [ ]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response


class Candidate_user_generate:

  def __init__(self):
      # DB Connection Information
      self.connection_info = "host=147.47.200.145 dbname=teamdb5 user=team5 password=snugraduate port=34543" # PostgreSQL 연결 주소

  def connect_SQL(self): # DB에서 데이터를 불러옴
    # PostgreSQL 연결
    conn = psycopg2.connect(self.connection_info)

    try:
        # 테이블을 Pandas.Dataframe으로 추출 (user table, course table, enrollment table)
        self.user = pd.read_sql('SELECT id, username FROM auth_user ',conn)   # user table
        self.course = pd.read_sql('SELECT cid_int, cid, cname FROM check_course',conn)  # check_course table
    ################################################################################################ 12.02 max_up id 쿼리 수정
        self.enroll = pd.read_sql('SELECT cid_int, user_id, cid from rec_enrollment where up_id in (select max(re.up_id) from rec_enrollment re group by re.user_id)',conn)  # enroll table  # exp -> rec # 최신데이터만 가져오는 부분이 있어야 할듯 (user_id별로 가장 큰 up_id만 가져온다거나)
    ################################################################################################ 
        self.enroll_idx = self.enroll.set_index('user_id')

    except psycopg2.Error as e:
        # 데이터베이스 에러 처리
        print("DB error: ", e)
        
    finally:
        # 데이터베이스 연결 해제 필수!! 
        conn.close()


  def connect_Neo4j(self, id, cnt): # 조건 쿼리, input : id - User ID / cnt : 몇명을 리턴할것인가? (default = 3)
    # DB Connection Information
    dbname = "teamdb5"
    uri_param = "bolt://147.47.200.145:37687"
    user_param = "team5"
    pwd_param = "snugraduate"
    apoc_string = 'CALL apoc.load.jdbc("jdbc:postgresql://localhost:34543/teamdb5?user=team5&password=snugraduate",'

    # Neo4j 연결
    conn = Neo4jConnection(uri=uri_param, user=user_param, pwd=pwd_param)
    # Cypher 쿼리 입력
    ################################################################################################     12.02 up_id 쿼리문 변경
    cypher = (apoc_string + 
              '"SELECT cid_int, user_id, cid, up_id, gpa from rec_enrollment where up_id IN (select max(re.up_id) from rec_enrollment re group by re.user_id)") YIELD row '
              'MERGE (sd:Student {id: row.user_id, up_id: row.up_id}) '                     # User Node 형성
              'MERGE (c:Course {cid: row.cid}) '  # 과목 Node 형성
              'MERGE (sd)-[e:Enrollment {gpa : row.gpa}]->(c) '  # 수강 Flow 형성
              'with sd,c '                                         
              'Match (sd{id:'+ str(id) + '})-->(c)<--(sd2) '         # User과 들은 과목을 들은 sd2를 찾아라
              'return distinct sd2.id, count(*) as cnt '              # sd2의 user id 반환
              'order by cnt desc '                                   # 겹치는 과목의 순서가 많은 사람부터
              'Limit '+ str(cnt)                                     # cnt명 반환
              )
    ################################################################################################ 
    # Cypher 쿼리 실행 후 결과를 print
    response = conn.query(cypher, db=dbname)

    #모든 노드 삭제 - 초기화 필수!!
    conn.query('MATCH (n) DETACH DELETE n', db=dbname)    

    # 연결 종료 필수!
    conn.close()
    
    # user_list = list(self.enroll_idx['cid'].loc[id]) # User가 들은 과목

    # 예외 처리 : 유사한 사용자가 없으면 전체 과목 리턴
    if response == None:
    ################################################################################################
      result_list = list(self.course['cid']) # 전체 과목 id list
      for value in list(self.enroll_idx['cid'].loc[id]): # User가 들은 과목 제외
        if value in result_list: # 조건추가 (혜령): user가 들은 과목중에 cid_list에 없는 과목이 있을 수도 있으므로 체크 필요
          result_list.remove(value)
      return result_list
    ################################################################################################      
      
    person = pd.DataFrame([dict(record) for record in response])
    # # print(person) # sd id 확인용
    ############################################################################################### 12.02 모집 인원수가 적어 해당 조건 삭제
    # # 예외 처리 : 특정 인원수가 안되면 그냥 전체 과목을 리턴
    # if person.shape[0] < cnt : 
    # ################################################################################################ 
    #   result_list = list(self.course['cid']) # 전체 과목 id list
    #   for value in list(self.enroll_idx['cid'].loc[id]): # # 조건추가 (영훈) : DS 필수과목 제외
    #     if value in result_list: # 조건추가 (혜령): user가 들은 과목중에 cid_list에 없는 과목이 있을 수도 있으므로 체크 필요
    #       result_list.remove(value)
    #   return result_list
    # ################################################################################################
    ############################################################################################### 

    # dictionray - 유저 확인용
    # cid_list = {}
    # for idx,i in person['sd2.id'].iteritems():
    #   cid_list[i] = list(self.enroll_idx['cid_int'].loc[i])
    # return cid_list

    cid_list = []
    for idx,i in person['sd2.id'].iteritems():
      cid_list+=list(self.enroll_idx['cid'].loc[i]) # 비슷한 과목을 들은 유저들의 과목을 리스트화
    cid_set = set(cid_list) # 중복 제거
    cid_list = list(cid_set) # 중복 제거

    for value in list(self.enroll_idx['cid'].loc[id]): # list에서 user가 들은 과목 제거
      if value in cid_list: # 조건추가 (혜령): user가 들은 과목중에 cid_list에 없는 과목이 있을 수도 있으므로 체크 필요
        cid_list.remove(value)
        

    # for value in check_list: # # 조건추가 (영훈) : DS 필수과목 제외
    #   if value in cid_list: # 조건추가 (혜령): user가 들은 과목중에 cid_list에 없는 과목이 있을 수도 있으므로 체크 필요
    #     cid_list.remove(value)
        

    # cid_list=[]
    # for i in cid_int_list:
    #   cid_list.append(self.course.set_index('cid_int')['cid'].loc[i])
    # cid_list

    # cid_list.sort() # 오름차순 정렬

    return cid_list

  def get_person(self, id, cnt=3):  # 3명의 비슷한 유저를 찾아라
    self.connect_SQL()  # DB에서 데이터 로드
    return self.connect_Neo4j(id, cnt) # Candidate cnt명을 찾고 그들의 과목 cid_int 반환

In [ ]:
import psycopg2
import psycopg2.extras as extras

# 연결정보 입력
connection_info = "host=147.47.200.145 dbname=teamdb5 user=team5 password=snugraduate port=34543"

# PostgreSQL 연결
# conn = psycopg2.connect(connection_info)

def select_from(conn, query):

  try:
      df = pd.read_sql(query,conn)

  except Exception as e:
      df = pd.DataFrame()
      print("Error: ", e)
      
  finally:
      conn.close()
      return df

In [ ]:
class Recommender():
  def __init__(self, 
               file_dir = './', 
               tfidf_fname = 'tfidf_tdm.pkl', 
               cid_fname = 'cid_list.pkl'):
    self.tfidf = joblib.load(os.path.join(file_dir, tfidf_fname))
    self.cid_list = joblib.load(os.path.join(file_dir, cid_fname))
    ################################################################################################
    self.basecourse_list = [   # DS 필수과목 리스트
    'M3239.000100',
    'M3239.000200',
    'M3239.000300',
    'M3239.003800',
    'M3239.005000',
    'M3239.005100',
    'M3239.005300',
    'M3239.005400',
    'M3239.005500',
    'M3239.005700',
    'M3239.006700',
    'M3239.002800',
    'M3239.003000',
    'M3239.000400',
    'M3239.000500',
    'M3239.000900',
    'M3239.005800',
    'M3239.003300'
    ]
    ################################################################################################
  def get_content_based_candidates(self, 
                                   grade_df,
                                   corpus = None,
                                   k = 10):
      
      # generate user representations
      tfidf_list = []
      for cid in grade_df['cid']:
        idx = np.where(self.cid_list==cid)[0]
        if len(idx) == 0: # 'M3239.002000' 데이터사이언스 대학원논문연구 를 포함해 cid_list에 없는 과목 존재
          #print(f"{cid} not in list")
          continue
        tfidf_list.append(self.tfidf[idx,:].toarray())
      user_embed = np.mean(np.stack(tfidf_list), axis=0)
        
      # calculate cosine similarity
      item_embed = self.tfidf
      cosine_sim = cosine_similarity(user_embed, item_embed)[0]

      # filter cid_list based on similarity, corpus, and whether already taken
      sim_scores = list(zip(self.cid_list, cosine_sim))
    ################################################################################################
      sim_scores = [x for x in sim_scores if (x[0] not in grade_df['cid'].tolist()) and (x[1] > 0) and (x[0] not in self.basecourse_list) ] # drop indices of already taken courses # drop courses with 0 similarity
    ################################################################################################
      if corpus is not None:
        corpus_intersect = set(self.cid_list).intersection(set(corpus))
        if len(corpus_intersect) > 0:
          sim_scores = [x for x in sim_scores if x[0] in corpus_intersect]
      
      # return top k
      sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
      sim_scores = sim_scores[:k]
      return sim_scores

  def get_user_based_candidates(self,
                                user_id,
                                grade_df,
                                k=10,
                                cnt=3):
    generator=Candidate_user_generate() # Class Instance 선언
    # cid_list 생성
    try:
      generated_list=generator.get_person(user_id, cnt=cnt) # input = user id , output = cid_int list
    except psycopg2.Error as e:
      try:
        generated_list=generator.get_person(user_id, cnt=cnt) # input = user id  # 커넥팅 에러 방지차원으로 한번 더 실행
      except psycopg2.Error as e:
        try:
          generated_list=generator.get_person(user_id, cnt=cnt) # input = user id  # 커넥팅 에러 방지차원으로 한번 더 실행
        except psycopg2.Error as e:
          # 데이터베이스 에러 처리
          print("DB error: ", e)
    return self.get_content_based_candidates(grade_df, generated_list, k=k)

  def get_recommendation(self, user_id, grade_df, include_undergrad=True, k=10, similar_user_cnt=3):
    candidates_scores_content = rec.get_content_based_candidates(grade_df, k=k)
    candidates_scores_user = rec.get_user_based_candidates(user_id, grade_df, k=k, cnt=similar_user_cnt)

    rec_scores = set(candidates_scores_content + candidates_scores_user)
    rec_list = sorted(rec_scores, key=lambda x: x[1], reverse=True)
    rec_list = [x[0] for x in rec_list]

    connection_info = "host=147.47.200.145 dbname=teamdb5 user=team5 password=snugraduate port=34543"
    conn = psycopg2.connect(connection_info)
    query2 = "SELECT a.cid, a.cname, a.program, a.classification, b.crd, a.dept, a.lang, a.yr_sem, a.last_yr_sem FROM (SELECT * FROM rec_courses WHERE cid IN (%s)) a LEFT JOIN check_course b ON a.cid=b.cid;" % str(rec_list)[1:-1]
    df = select_from(conn, query2)
    df = df.set_index('cid').loc[rec_list].reset_index()
    if not include_undergrad: 
      df = df.loc[df['program']=='대학원'].reset_index(drop=True)
    return df

In [ ]:
# uid3_grade_df = pd.DataFrame({'cid': ['M3239.000100',
#  'M3239.000200',
#  'M3239.000300',
#  'M3239.003300',
#  'M3224.000100',
#  'M3239.002400',
#  'M3239.003500',
#  'M3239.003800',
#  'M3239.001000',
#  'M3239.003300',
#  'M3239.005200',
#  'M3239.002000',
#  'M3239.003300',
#  'M3239.004000']})

In [ ]:
uid = 2
connection_info = "host=147.47.200.145 dbname=teamdb5 user=team5 password=snugraduate port=34543"
conn = psycopg2.connect(connection_info)
user_grade_df = select_from(conn,'SELECT cid_int, user_id, cid from rec_enrollment where up_id = (select max(re.up_id) from rec_enrollment re where user_id = %s)' %uid)

In [ ]:
# %%timeit
rec = Recommender()
################################################################################################
rec.get_recommendation(uid, user_grade_df)
################################################################################################

,cid,cname,program,classification,crd,dept,lang,yr_sem,last_yr_sem
0,430.659,컴퓨터 및 VLSI특강,대학원,전선,3,공과대학_전기·정보공학부,"한국어, 영어","2020-1학기, 2020-2학기, 2021-1학기, 2021-2학기, 2022-1...",2022-2학기
1,M3500.000300,(공유)빅데이터 개론 1,학사,전선,3,혁신공유학부_혁신공유학부,한국어,2022-1학기,2022-1학기
2,406.561,데이터기반학습,대학원,전선,3,공과대학_산업공학과,한국어,"2020-1학기, 2021-1학기",2021-1학기
3,M1522.003900,인공지능 및 빅데이터시스템,대학원,전선,3,공과대학_컴퓨터공학부,한국어,2021-1학기,2021-1학기
4,M2608.001300,기계학습 기초 및 전기정보 응용,학사,전선,3,공과대학_전기·정보공학부,한국어,"2020-1학기, 2021-1학기, 2022-1학기",2022-1학기
5,M2177.003100,딥러닝,대학원,전선,3,"공과대학_전기·정보공학부, 공과대학_조선해양공학과",한국어,"2020-2학기, 2021-2학기, 2022-2학기",2022-2학기
6,L0444.000800,컴퓨팅 응용: 데이터사이언스의 기초,학사,교양,3,기초교육원,한국어,2022-2학기,2022-2학기
7,L0444.000600,컴퓨팅 응용: 기계학습 개념 및 실습,학사,교양,3,기초교육원,한국어,2022-1학기,2022-1학기
8,4190.428,기계학습 개론,학사,전선,3,공과대학_컴퓨터공학부,한국어,2021-2학기,2021-2학기
9,M3380.000300,의료기기와 인공지능,대학원,전선,3,의과대학_의료기기산업학과,한국어,2022-1학기,2022-1학기


In [ ]:
# %%timeit
################################################################################################
rec.get_recommendation(uid, user_grade_df, include_undergrad=False)
################################################################################################

,cid,cname,program,classification,crd,dept,lang,yr_sem,last_yr_sem
0,430.659,컴퓨터 및 VLSI특강,대학원,전선,3,공과대학_전기·정보공학부,"한국어, 영어","2020-1학기, 2020-2학기, 2021-1학기, 2021-2학기, 2022-1...",2022-2학기
1,406.561,데이터기반학습,대학원,전선,3,공과대학_산업공학과,한국어,"2020-1학기, 2021-1학기",2021-1학기
2,M1522.003900,인공지능 및 빅데이터시스템,대학원,전선,3,공과대학_컴퓨터공학부,한국어,2021-1학기,2021-1학기
3,M2177.003100,딥러닝,대학원,전선,3,"공과대학_전기·정보공학부, 공과대학_조선해양공학과",한국어,"2020-2학기, 2021-2학기, 2022-2학기",2022-2학기
4,M3380.000300,의료기기와 인공지능,대학원,전선,3,의과대학_의료기기산업학과,한국어,2022-1학기,2022-1학기
5,M3239.004000,자연어처리의 응용,대학원,전선,3,데이터사이언스대학원_데이터사이언스학과,한국어,"2021-1학기, 2022-1학기, 2022-2학기",2022-2학기
6,M3239.002300,데이터사이언스 특강,대학원,전선,3,데이터사이언스대학원_데이터사이언스학과,"한국어, 영어","2020-2학기, 2021-1학기, 2021-2학기, 2022-1학기",2022-1학기
7,M3239.002400,엠비언트 인공지능 플랫폼 및 실습,대학원,전선,3,데이터사이언스대학원_데이터사이언스학과,한국어,"2020-2학기, 2021-2학기",2021-2학기
8,M3239.003100,데이터 분석과 시각화,대학원,전선,3,데이터사이언스대학원_데이터사이언스학과,"한국어, 영어","2020-2학기, 2021-2학기, 2022-2학기",2022-2학기
9,M3239.005200,머신러닝의 수리적 입문,대학원,전선,3,데이터사이언스대학원_데이터사이언스학과,한국어,2022-1학기,2022-1학기


In [ ]:
course_all = pd.read_excel('../Data/rec_courses/courses_final.xlsx')

In [ ]:
course_all.loc[course_all['cid'].isin(user_grade_df['cid'])]

,cid,program,cname,avg_quota,latest_classification,department,language,year_semesters,latest_year_semester,notes
3755,M3239.000100,대학원,빅데이터 및 지식 관리 시스템,50.000000,전선,데이터사이언스대학원_데이터사이언스학과,한국어,"2020-1학기, 2021-1학기",2021-1학기,타과생의 경우 수강정정기간에 수강신청 가능함
3756,M3239.000200,대학원,데이터사이언스를 위한 머신러닝 및 딥러닝,60.000000,전선,데이터사이언스대학원_데이터사이언스학과,영어,"2020-1학기, 2021-1학기",2021-1학기,"ⓔ, ⓔ타과생의 경우 수강정정기간에 수강신청 가능함"
3757,M3239.000300,대학원,데이터 사이언스를 위한 소프트웨어 플랫폼,90.000000,전선,데이터사이언스대학원_데이터사이언스학과,한국어,"2020-1학기, 2021-1학기",2021-1학기,타과생의 경우 수강정정기간에 수강신청 가능함
3760,M3239.000900,대학원,데이터사이언스 세미나 특강,40.000000,전선,데이터사이언스대학원_데이터사이언스학과,한국어,2020-1학기,2020-1학기,타과생의 경우 수강정정기간에 수강신청 가능함
6537,M3224.000100,대학원,시각적 이해를 위한 기계학습,60.000000,전선,데이터사이언스대학원_데이터사이언스학과,"한국어, 영어","2020-2학기, 2021-1학기, 2021-2학기",2021-2학기,ⓔ
6542,M3239.001000,대학원,데이터사이언스 캡스톤 프로젝트,11.470588,전선,데이터사이언스대학원_데이터사이언스학과,"한국어, 영어","2020-2학기, 2021-1학기, 2021-2학기, 2022-1학기, 2022-2학기",2022-2학기,"ⓔ, ⓔ® 타과생 수강 불허, ®, ®타과생 수강 불가, ®타과생 수강 불허, ®타..."
6543,M3239.001100,대학원,텍스트 및 자연어 빅데이터 분석 방법론,50.000000,전선,데이터사이언스대학원_데이터사이언스학과,한국어,"2020-2학기, 2021-2학기",2021-2학기,ⓒ
6963,M3239.004100,대학원,데이터사이언스와 강화학습,53.333333,전선,데이터사이언스대학원_데이터사이언스학과,영어,"2021-1학기, 2021-2학기, 2022-1학기",2022-1학기,ⓔ
